# Importes

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import slate3k as slate
import nltk
from lxml import etree
from urllib.request import urlopen

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/p1ntsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Funções e classes

In [3]:
class RequestBeautifulSoup:
    def __init__(self,url):
        self.url = url
        self.intervalos = None
        self.perguntas = None
        self.respostas = None
    
    # Request do soup da biblioteza BeautifulSoup
    def requestSoup(self):
        response = get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    # Intervalos de onde estão as respostas
    def getIntervalos(self,listaPrincipal):
        intervalos = []
        for i in range(len(self.perguntas[1])):
            if(i != len(self.perguntas[1])-1):
                intervalos.append((self.perguntas[1][i]+1,self.perguntas[1][i+1]))
            else:
                intervalos.append((self.perguntas[1][i]+1,len(listaPrincipal)))
        self.intervalos = intervalos

    # Perguntas e suas posições
    def getPerguntas(self,soup, chaves = None):
        perguntas = []
        posi = []
        for i in range(len(soup)):
            if("?" in soup[i].text):
                if(chaves is not None):
                    texto = soup[i].text
                    for j in chaves:
                        texto = texto.replace(j,"")
                    perguntas.append(texto)
                    posi.append(i)
                else:
                    perguntas.append(soup[i].text)
                    posi.append(i)

        self.perguntas = [perguntas,posi]
    
    # Respostas
    def getRespostas(self,soup,chaves = None,subs = ""):
        respostas = []
        for i in self.intervalos:
            text = ""
            for j in range(i[0],i[1]):
                text += soup[j].text
            if(chaves is not None):
                for j in chaves:
                    text = text.replace(j,subs)
                respostas.append(text)
            else:
                respostas.append(text)
        self.respostas = respostas
        
    def getComparacao(self,first,second):
        if(len(first) == len(second)):
            return True
        else:
            return False
        
    def criarDataframePDF(self,perguntas,respostas,salve = None):
        if(salve is not None):    
            pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./' + salve + '.csv', index= False)
        else:
            return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})
    

In [37]:


def getXpathWay(url):
    response = urlopen(url)
    htmlparser = etree.HTMLParser()
    return etree.parse(response, htmlparser)
        

def getPerguntasXpath(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i]):
            if(chaves is not None):
                texto = soup[i]
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i])
                posi.append(i)
                
    return [perguntas,posi]
                
def getPerguntasPDF(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i]):
            if(chaves is not None):
                texto = soup[i]
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i])
                posi.append(i)
    return [perguntas,posi]

def getRespostasPDF(soup,intervalos,chaves = None):
    respostas = []
    for i in intervalos:
        text = ""
        for j in range(i[0],i[1]):
            text += soup[j]
        if(chaves is not None):
            for k in chaves:
                text = text.replace(k,"")
        respostas.append(text)
    return respostas

# Intervalos de onde estão as respostas
def getIntervalos(posicoes,listaPrincipal):
    intervalos = []
    for i in range(len(posicoes)):
        if(i != len(posicoes)-1):
            intervalos.append((posicoes[i]+1,posicoes[i+1]))
        else:
            intervalos.append((posicoes[i]+1,len(listaPrincipal)))
    return intervalos

def criarDataframePDF(perguntas,respostas,salve = None):
    if(salve is not None):    
        pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./' + salve + '.csv', index= False)
    else:
        return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})

# Código

In [166]:
with open('./PR.html','r') as f:
    objeto_soup_1 = BeautifulSoup(f, 'html.parser')

In [96]:
objeto_soup = RequestBeautifulSoup('https://www.saude.rj.gov.br/noticias/2020/03/perguntas-e-respostas-sobre-o-novo-coronavirus')
soup = objeto_soup.requestSoup()

SSLError: HTTPSConnectionPool(host='www.saude.rj.gov.br', port=443): Max retries exceeded with url: /noticias/2020/03/perguntas-e-respostas-sobre-o-novo-coronavirus (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))

In [167]:
objeto_soup_1

<div class="field field--name-bp-text field--type-text-long field--label-hidden field--item"><p><a id="perguntas"></a></p>
<p><a class="btn more" href="#o-que-e">O que é</a> <a class="btn more" href="#sintomas">Sintomas </a> <a class="btn more" href="#contagio">Transmissão</a> <a class="btn more" href="#fezes">Transmissão por fezes</a> <a class="btn more" href="#vacina">A vacina</a> <a class="btn more" href="#higiene">Higiene</a> <a class="btn more" href="#outros">Outros coronavirus</a> <a class="btn more" href="#incoubacao">Incubação</a> <a class="btn more" href="#mascaras">Máscaras</a> <a class="btn more" href="#superficies">Tempo nas superfícies</a> <a class="btn more" href="#pacote">Pacotes</a> <a class="btn more" href="#trabalho">Local de trabalho</a> <a class="btn more" href="#caseiro">Gel caseiro</a> <a class="btn more" href="#fumantes">Fumantes</a> <a class="btn more" href="#hiv">HIV</a> <a class="btn more" href="#animal">Contágio e animais</a> <a class="btn more" href="#cloroq

In [168]:
div_soup = objeto_soup_1.find('div', class_= "field field--name-bp-text field--type-text-long field--label-hidden field--item")
list_soup = div_soup.find_all(['p','ul','il','a','h2','h3','h4'])
#list_soup = objeto_soup_1.find_all(['div'])

In [172]:
list_soup[22:]

[<p><strong>O que é COVID-19?</strong><br/>
 COVID-19 é a doença infecciosa causada pelo novo coronavírus, identificado pela primeira vez em dezembro de 2019, em Wuhan, na China.</p>,
 <p><strong>Existe uma vacina contra COVID-19?</strong><br/>
 Sim. As autoridades reguladoras nacionais estão revisando algumas vacinas contra COVID-19 e uma já foi aprovada em alguns países. Estudos abrangentes sobre várias vacinas candidatas têm relatado resultados preliminares encorajadores. É provável que outras candidatas a vacina sejam submetidas às autoridades regulatórias para aprovação.</p>,
 <p>Uma vez que se comprove que uma ou mais vacinas são seguras e eficazes, elas devem ser aprovadas pelas autoridades regulatórias nacionais, fabricadas de acordo com os padrões exigidos e distribuídas. A OMS e a OPAS estão trabalhando com parceiros em todo o mundo para ajudar a coordenar as principais etapas desse processo, incluindo a facilitação do acesso equitativo a vacinas seguras e eficazes contra COV

In [133]:
list_soup = list_soup[0].find_all(['div'])

In [173]:
list_soup = list_soup[22:]

In [135]:
text = ""
for i in list_soup:
    #print(i.text)
    text += i.text

In [174]:
list_soup = [i.text for i in list_soup]

In [175]:
list_soup

['O que é COVID-19?\nCOVID-19 é a doença infecciosa causada pelo novo coronavírus, identificado pela primeira vez em dezembro de 2019, em Wuhan, na China.',
 'Existe uma vacina contra COVID-19?\nSim. As autoridades reguladoras nacionais estão revisando algumas vacinas contra COVID-19 e uma já foi aprovada em alguns países. Estudos abrangentes sobre várias vacinas candidatas têm relatado resultados preliminares encorajadores. É provável que outras candidatas a vacina sejam submetidas às autoridades regulatórias para aprovação.',
 'Uma vez que se comprove que uma ou mais vacinas são seguras e eficazes, elas devem ser aprovadas pelas autoridades regulatórias nacionais, fabricadas de acordo com os padrões exigidos e distribuídas. A OMS e a OPAS estão trabalhando com parceiros em todo o mundo para ajudar a coordenar as principais etapas desse processo, incluindo a facilitação do acesso equitativo a vacinas seguras e eficazes contra COVID-19 para bilhões de pessoas que delas precisarão.',
 '

In [107]:
list_soup = ['O que é coronavírus?',
 'Coronavírus é uma família de vírus que causam infecções respiratórias. O mais novo, COVID-19, foi descoberto ano passado na China, mas outros tipos de coronavírus já foram isolados desde 1937 e há mais quatro conhecidos.','Como o coronavírus é transmitido?',
 'A transmissão do coronavírus de pessoa para pessoa acontece pelo ar com secreções respiratórias, como gotículas de saliva, espirro, tosse, catarro; ou contato próximo com superfícies contaminadas, como toque em objetos ou aperto de mão. A transmissão pode ocorrer mesmo sem o aparecimento de sinais e sintomas.','Quais os sintomas?',
 'É um vírus com os sintomas muito semelhantes ao da gripe, como febre, queda do estado geral, tosse, coriza e sintomas respiratórios.','Como é feito o diagnóstico do coronavírus no Rio de Janeiro?',
 'A Vigilância municipal coleta uma amostra de material respiratório do paciente suspeito e a entrega ao Laboratório Central Noel Nutels (Lacen-RJ). No Lacen, o material é dividido em duas partes. Uma parte da amostra é analisada na própria unidade, investigando vírus respiratórios comuns, e a outra é enviada à Fiocruz, que realiza a testagem para o coronavírus. Somente o resultado final do Lacen descarta ou confirma os casos.Os casos leves devem são acompanhados pelas Vigilâncias municipais, acionando as medidas de precaução domiciliar. Já os casos graves são encaminhados a um hospital de referência para isolamento e tratamento.','Como é o tratamento do coronavírus?',
 'Não existe tratamento específico para o coronavírus COVID-19. São indicados repouso e consumo de bastante água, além de algumas medidas adotadas para aliviar os sintomas.','Se eu já tive coronavírus, posso ter de novo?',
 'Os dados mais recentes indicam, até agora, que quem teve uma vez o coronavírus do tipo COVID-19, que é o que está circulando nos países atualmente, não está sendo contaminado novamente. Mas o vírus ainda está sendo estudado.','Estou me sentindo doente. O que devo fazer?',
 'Se você tem sintomas de gripe ou resfriado, mas apresenta bom estado geral, fique em casa por 14 dias para evitar a contaminação de outras pessoas. Faça repouso e siga as medidas de higiene para',
 'reduzir o risco aos seus familiares.',
 'Se você tem piora do estado geral, cansaço ou dificuldade para respirar, procure uma emergência. O médico decidirá se você necessita de internação ou exame para o coronavírus.',
 'Bebês e crianças menores de seis anos, gestantes, mulheres que deram à luz há menos de 40 dias, maiores de 60 anos, e pessoas com doenças preexistentes devem ser avaliadas por profissional de',
 'saúde caso apresentem febre e sintomas respiratórios.','Preciso sair de casa. Devo usar máscara de proteção?',
 'Se apresentar sintomas é recomendado o isolamento domiciliar por 14 dias. Se for realmente necessário sair, a orientação é usar máscaras de proteção e evitar ficar próximo de outras pessoas para evitar uma possível transmissão no seu trânsito. Quem tiver contato com casos suspeitos, incluindo profissionais de saúde e familiares, também devem usar máscaras.','Como se prevenir do coronavírus?',
 '- Lavar as mãos frequentemente com água e sabonete por pelo menos 20 segundos, incluindo o',
 'espaço entre os dedos, unhas e punhos;',
 '- Se não houver água e sabonete, usar um desinfetante para as mãos à base de álcool;',
 '- Evitar tocar nos olhos, nariz e boca com as mãos não lavadas;',
 '- Evitar contato próximo com pessoas doentes;',
 '- Ficar em casa quando estiver doente;',
 '- Cobrir boca e nariz ao tossir ou espirrar com um lenço de papel e jogar no lixo; se não tiver, tampar nariz e boca com a parte interna do cotovelo, nunca usar as mãos;',
 '- Limpar e desinfetar objetos e superfícies tocados com frequência, como celulares;',
 '- Profissionais de saúde devem utilizar medidas de precaução padrão, de contato e de gotículas',
 '(máscara cirúrgica, luvas, avental não estéril e óculos de proteção).Posso usar qualquer tipo de álcool, ou precisa ser em gel? Tem que ser 70?',
 'Estudos mostram que o álcool 60% já alcança ação antimicrobiana, contra bactérias, fungos e vírus.',
 'Pode ser em gel ou o líquido, vendido em mercados, mas precisa ser 60% ou mais. No entanto, o segundo resseca as mãos e tem risco de acidentes aumentado. O álcool com teor abaixo de 60% não é eficaz.','A que distância preciso estar das pessoas para não me contaminar e não contaminar ninguém?',
 'O ideal é manter uma distância superior a 2 metros e evitar estar frente a frente quando não for possível essa distância.','Que tipo de produto devo usar para limpar a casa?',
 'Pode usar desinfetante ou álcool 60% ou mais, além de solução com água sanitária.','Quais são as medidas adotadas nas aeronaves que chegam do exterior?',
 'A identificação de pessoas com sinais e sintomas respiratórios para testagem e adoção das medidas cabíveis caso o resultado seja positivo.',
 ]

In [125]:
list_soup = [i for i in text.split('\n') if i != ' ' or i != '']

In [160]:
perguntas = getPerguntasPDF(list_soup,['\xa0','\n','\t','\r'])

In [161]:
intervalos = getIntervalos(perguntas[1],list_soup)

In [162]:
respostas = getRespostasPDF(list_soup,intervalos,['\t','\r'])

In [163]:
len(perguntas[0]) == len(respostas)

True

In [165]:
criarDataframePDF(perguntas[0],respostas,'/www.hospitalsaodomingos.com.br')

In [77]:
objeto_soup.getPerguntas(list_soup,['\xa0','\n'])
objeto_soup.getIntervalos(list_soup)
objeto_soup.getRespostas(list_soup,['\xa0','\n'])

In [78]:
print(objeto_soup.perguntas[0])
print(objeto_soup.respostas)
print(objeto_soup.getComparacao(objeto_soup.perguntas[0],objeto_soup.respostas))

['Como prevenir?As orientações do Ministério da Saúde sobre a prevenção de contágio são: lavar as mãos com água e sabão ou usar álcool em gel; ao tossir ou espirrar cobrir o nariz e a boca com um lenço ou braço, não utilizar as mãos; manter ambientes bem ventilados; e não compartilhar objetos pessoais; reduzir o contato social.', 'Quem deve ir ao hospital?Só deve ir ao hospital quem estiver se sentindo muito mal ou apresentar sintomas como febre alta e dificuldade para respirar. Se for só um resfriado ou gripe leve, a recomendação é ficar em casa. De acordo com infectologistas, ir ao hospital, além de sobrecarregar o sistema, aumenta o risco de a pessoa contrair uma infecção.', 'Quais são os sintomas associados a pandemia?Os sintomas do Covid-19 envolvem febre, cansaço e tosse seca. Alguns pacientes podem apresentar dores, congestão nasal, coriza, tosse e diarreia. Enquanto outros pacientes podem ser assintomáticos, ou seja, estarem infectados pelo vírus, mas não apresentarem sintomas.

In [57]:
objeto_soup.criarDataframePDF(objeto_soup.perguntas[0],objeto_soup.respostas,'coronavirus.rj.gov.br')

In [87]:
with open('../img/1.pdf','rb') as f:
    extract = slate.PDF(f)[0]

In [88]:
extract.split('\n\n')

['Algumas pessoas são infectadas mas não desenvolvem sintoma algum e não se sentem mal. A ',
 'maioria das pessoas (cerca de 80%) se recupera da doença sem a necessidade de nenhum ',
 'tratamento especial. Os casos mais graves desenvolvem dificuldade de respirar.  ',
 'Pessoas idosas, aquelas com o sistema imunológico enfraquecido ou problemas de saúde ',
 'preexistentes, como hipertensão, problemas cardíacos ou diabetes, estão mais suscetíveis a ',
 'adoecer gravemente. Quem \natendimento médico. ',
 'tiver ',
 'febre, ',
 'tosse e dificuldade para ',
 'respirar deve procurar ',
 'O COVID-19 é um vírus novo e não se sabe ao certo como interage com Chagas, pois não temos ',
 'evidências científicas suficientes. É certo que aqueles que já têm problemas cardíacos graves ',
 'causados pela doença de Chagas ou um sistema imunológico enfraquecido correm mais risco ',
 'de desenvolver sintomas mais graves de COVID-19. ',
 'Se sou paciente crônico de doença de Chagas sem complicações graves, 

In [84]:
perguntas = ['Quais são os sintomas de COVID-19? São diferentes para um paciente de doença de Chagas?','Se sou paciente crônico de doença de Chagas sem complicações graves, sou mais vulnerável ao COVID-19?','Se estou na fase aguda da doença de Chagas e tenho sintomas de COVID- 19, o que devo fazer?','Se eu tiver sintomas ou o resultado do teste de COVID-19 for positivo, preciso interromper o tratamento de doença de Chagas?','O que devo fazer se eu tiver doença de Chagas sem tratamento específico e tiver sintomas de COVID-19?','O que devo fazer para proteger minha família e a mim do COVID-19 se tenho doença de Chagas?','Se eu tiver complicações cardiológicas ou digestivas causadas pela doença de Chagas, o que preciso fazer para me proteger do COVID-19?','Devo continuar frequentando o centro de saúde para dar continuidade ao tratamento de Chagas durante a epidemia de COVID-19?','Se eu receber um resultado positivo para COVID-19 e tiver doença de Chagas, quais são os tratamentos que não devo tomar?','Se estou grávida e suspeito que tenha contraído doença de Chagas, sou mais vulnerável ao COVID-19?','Se eu estiver grávida, tiver doença de Chagas e receber um resultado positivo para COVID-19, o que devo fazer?']

In [85]:
perguntas

['Quais são os sintomas de COVID-19? São diferentes para um paciente de doença de Chagas?',
 'Se sou paciente crônico de doença de Chagas sem complicações graves, sou mais vulnerável ao COVID-19?',
 'Se estou na fase aguda da doença de Chagas e tenho sintomas de COVID- 19, o que devo fazer?',
 'Se eu tiver sintomas ou o resultado do teste de COVID-19 for positivo, preciso interromper o tratamento de doença de Chagas?',
 'O que devo fazer se eu tiver doença de Chagas sem tratamento específico e tiver sintomas de COVID-19?',
 'O que devo fazer para proteger minha família e a mim do COVID-19 se tenho doença de Chagas?',
 'Se eu tiver complicações cardiológicas ou digestivas causadas pela doença de Chagas, o que preciso fazer para me proteger do COVID-19?',
 'Devo continuar frequentando o centro de saúde para dar continuidade ao tratamento de Chagas durante a epidemia de COVID-19?',
 'Se eu receber um resultado positivo para COVID-19 e tiver doença de Chagas, quais são os tratamentos que n